# Galactic trade routes
You are the supreme ruler of the universe.

You want to establish trade routes between your planets, to make your empire more prosperous.

As an organized ruler, you have listed all your planets, their 3D coordinates, and their type. You identified two types: planets producing mainly food and needing metal, and planets producing mostly metal and needing food.

Every food planet should have a trade route with at least one metal planet, and every metal planet should have a trade route with at least one food planet.

Goods cannot transit on another planet after they arrive at their first destination. So obviously, connecting two planets of the same type is useless.

A planet can exchange goods with multiple other planets. Multiple planet clusters, isolated from one another, can exist if every planet has a trade route to a complementary planet.

How can you compute the trade routes to establish so that the sum of the distances is the lowest possible?

In [28]:
from typing import List

class Planet:
    def __init__(self,x:float,y:float,z:float,is_food:bool):
        self.x=x
        self.y=y
        self.z=z
        self.id = 0
        self.is_food=is_food
        self.is_connected=False
        self.connected = []
        
    def set_id(self,id:int):
        self.id = id
        
class GalaticEmpire:
    def __init__(self,name:str,planets:List[Planet]):
        self.name = name
        self.planets = []
        for planet in planets:
            planet.set_id(len(self.planets))
            self.planets.append(planet)
        self.trade_routes = []
        self.total_distance = 0
        
    def add_planet(self,planet:Planet):
        planet.set_id(len(self.planets))
        self.planets.append(planet)
        
    def establish_trade_routes(self):
        self.total_distance,self.trade_routes,self.planets = self.establish_trade_route(self.planets)
        return self.total_distance,self.trade_routes,self.planets
        
    def establish_trade_route(self,planets:List[Planet]):
        total_distance = 0
        trade_routes = []
        food_planets = [planet for planet in planets if planet.is_food]
        non_food_planets = [planet for planet in planets if not planet.is_food]
        # connect food planets to nearest non-food planets
        for food_planet in food_planets:
            distance, complementary = self.find_nearest_complementary(food_planet,non_food_planets)
            if complementary:
                total_distance += distance
                trade_routes.append((food_planet, complementary, distance))
                food_planet.connected.append(food_planet)
                complementary.connected.append(food_planet)
                food_planet.is_connected = True
                complementary.is_connected = True
                
        # connect remaining non-food planets to nearest non-connected food planets
        for non_food_planet in non_food_planets:
            if not non_food_planet.is_connected:
                distance, complementary = self.find_nearest_complementary(non_food_planet,food_planets)
                if complementary:
                    total_distance += distance
                    trade_routes.append((non_food_planet, complementary, distance))
                    non_food_planet.connected.append(non_food_planet)
                    complementary.connected.append(non_food_planet)
                    non_food_planet.is_connected = True
                    complementary.is_connected = True
                    
        # find remaining unconnected planets and try to connect them to nearest planets
        unconnected_planets = [planet for planet in planets if not planet.is_connected]
        for unnconected in unconnected_planets:
            p = unnconected
            distance, complementary = self.find_nearest_complementary(p,planets)
            if complementary:
                total_distance += distance
                trade_routes.append((p, complementary, distance))
                p.connected.append(complementary)
                complementary.connected.append(p)
                p.is_connected = True
                complementary.is_connected = True
                if complementary in unconnected_planets:
                    unconnected_planets.remove(complementary)
            else:
                print(f"No complementary planet found for planet {p.id}")
                
        return (total_distance,trade_routes,unconnected_planets)
        
    def print_trade_routes(self):
        print("Trade routes for",self.name)
        for route in self.trade_routes:
            print(route[0].x,route[0].y,route[0].z,route,"->",route[1].x,route[1].y,route[1].z,route,":",route[2])

    def get_distance(self,p1:Planet,p2:Planet):
        return ((p1.x-p2.x)**2+(p1.y-p2.y)**2+(p1.z-p2.z)**2)**0.5

    def find_nearest_complementary(self,p1:Planet,planets:List[Planet]):
        min_dist = float('inf')
        nearest = None
        for p2 in planets:
            if p2.is_food != p1.is_food and not p2.is_connected:
                distance = self.get_distance(p1,p2)
                if distance < min_dist:
                    min_dist = distance
                    nearest = p2
        return min_dist, nearest


In [29]:

planets = [Planet(0,0,0,True), Planet(1,1,1,False), Planet(2,2,2,True), Planet(3,3,3,False),Planet(4,4,4,True)]
empire = GalaticEmpire("Galactic Empire",planets)
distance,routes,unnconected = empire.establish_trade_routes()
print("Total distance:",distance)
print("Trade routes:")
for route in routes:
    print(route[0].x,route[0].y,route[0].z,f"({route[0].id})->({route[1].id})",route[1].x,route[1].y,route[1].z,": distance =",route[2])
print(f"Unconnected planets: {[x.id for x in unnconected]}")


No complementary planet found for planet 4
Total distance: 3.4641016151377544
Trade routes:
0 0 0 (0)->(1) 1 1 1 : distance = 1.7320508075688772
2 2 2 (2)->(3) 3 3 3 : distance = 1.7320508075688772
Unconnected planets: [4]
